#Biodenoising - Animal vocalization denoising
This is a demo for animal vocalization denoising without access to clean data.
For the more info check the [associated page](https://mariusmiron.com/research/biodenoising/) and the code repository on [github](https://github.com/earthspecies/biodenoising).

First, let's install the package from pip:

In [ ]:
!pip install biodenoising

We import the libraries:

In [ ]:
from IPython import display as disp
import os
import torch
import torchaudio
from biodenoising import pretrained
from biodenoising.denoiser.dsp import convert_audio

We download some noisy animal vocalizations from the biodenoising_validation dataset. Note that these files, species, noise conditions were not seen during training, to test for generalization.

In [ ]:
!wget https://storage.googleapis.com/esp-public-files/biodenoising/demo/benchmark/12_terrestrial_original.wav
!wget https://storage.googleapis.com/esp-public-files/biodenoising/demo/benchmark/14_terrestrial_original.wav
!wget https://storage.googleapis.com/esp-public-files/biodenoising/demo/benchmark/36_underwater_original.wav
!wget https://storage.googleapis.com/esp-public-files/biodenoising/demo/benchmark/30_terrestrial_original.wav
!wget https://storage.googleapis.com/esp-public-files/biodenoising/demo/benchmark/34_terrestrial_original.wav
!wget https://storage.googleapis.com/esp-public-files/biodenoising/demo/benchmark/3_terrestrial_original.wav
!wget https://storage.googleapis.com/esp-public-files/biodenoising/demo/benchmark/21_terrestrial_original.wav
!wget https://storage.googleapis.com/esp-public-files/biodenoising/demo/benchmark/52_underwater_original.wav
!wget https://storage.googleapis.com/esp-public-files/biodenoising/demo/benchmark/24_underwater_original.wav

We set the device, gpu or cpu. You can use a computing instance with a GPU for faster processing.

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

Let's load the 16kHz model. If it's the first time you run this, it will download the model locally.

In [ ]:
model = pretrained.biodenoising16k_dns48().to(device)

We use the model above to denoise the first demo sound.

In [ ]:
wav, sr = torchaudio.load(os.path.join('12_terrestrial_original.wav'))
wav = convert_audio(wav, sr, model.sample_rate, model.chin).to(device)
with torch.no_grad():
    denoised = model(wav[None])[0]
disp.display(disp.Audio(wav.data.cpu().numpy(), rate=model.sample_rate))
disp.display(disp.Audio(denoised.data.cpu().numpy(), rate=model.sample_rate))

We now process the remaining files.

In [ ]:
file_list = ['36_underwater_original.wav','14_terrestrial_original.wav','3_terrestrial_original.wav','21_terrestrial_original.wav','24_underwater_original.wav','30_terrestrial_original.wav','34_terrestrial_original.wav']
for f in file_list:
  wav, sr = torchaudio.load(os.path.join(f))
  wav = convert_audio(wav, sr, model.sample_rate, model.chin).to(device)
  with torch.no_grad():
      denoised = model(wav[None])[0]
  print(f)
  disp.display(disp.Audio(wav.data.cpu().numpy(), rate=model.sample_rate))
  disp.display(disp.Audio(denoised.data.cpu().numpy(), rate=model.sample_rate))

You can download the whole benchmarking dataset from zenodo and process it (subfolder 16000/noisy). You can also check the clean vocalizations and the added noise.

In [ ]:
!wget https://zenodo.org/records/13736465/files/biodenoising_validation_1.0.zip
!unzip biodenoising_validation_1.0.zip